# Clustering Crypto

In [21]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [22]:
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [23]:
# Creating a DataFrame (which needed to be transposed)
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()


,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,FullName,FullyPremined,Id,...,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,154593,0,0,N/A,42 Coin,1427211129,42 Coin (42),0,4321,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,300 token (300),0,749869,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,365Coin (365),0,33639,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview
404,Scrypt,287687,0,60,N/A,404Coin,1466100361,404Coin (404),0,21227,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,602,False,404,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",532000000,N/A,0,/coins/404/overview
433,N/A,9191708,2,0,7605,433 Token,1541597321,433 Token (433),0,926547,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0x738505a5f31bf72e0b70298bca81150eb1b7c751,3505,False,433,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",1000000000,N/A,1.09091e+08,/coins/433/overview


### Data Preprocessing

In [24]:
# Remove unnecessary columns
slim_crypto_df = crypto_df.drop(columns=['Rating','TotalCoinsFreeFloat', 'Url', 'Taxonomy', 'SortOrder', 'Sponsored', 'SmartContractAddress', 'Id', 'BuiltOn', 'FullName', 'ImageUrl', 'ContentCreatedOn', 'PreMinedValue'])


In [25]:
# Keep only cryptocurrencies that are on trading
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['IsTrading'] == True]

In [26]:
# Keep only cryptocurrencies with a working algorithm
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['Algorithm'] != 'N/A']

In [27]:
# Remove the "IsTrading" column
slim_crypto_df = slim_crypto_df.drop(columns='IsTrading')


In [28]:
# Removing rows with at least 1 null value
slim_crypto_df = slim_crypto_df.dropna()


In [29]:
# Removing rows with cryptocurrencies without coins mined
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['TotalCoinsMined'] != 0]


In [30]:
# Dropping rows where there are 'N/A' text values
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['TotalCoinSupply'] != 'N/A']
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['ProofType'] != 'N/A']


In [31]:
# Fetching the cryptocurrencies names prior to drop them from crypto_df
names = slim_crypto_df['CoinName']


In [32]:
# Removig the cryptocurrency name since it's not going to be used on the clustering algorithm
slim_crypto_df = slim_crypto_df.drop(columns='CoinName')

In [33]:
#Change numerical data from objects to floats
slim_crypto_df['BlockNumber'] = slim_crypto_df.BlockNumber.astype(float)
slim_crypto_df['BlockReward'] = slim_crypto_df.BlockReward.astype(float)
slim_crypto_df['BlockTime'] = slim_crypto_df.BlockTime.astype(float)
slim_crypto_df['FullyPremined'] = slim_crypto_df.FullyPremined.astype(float)
slim_crypto_df['NetHashesPerSecond'] = slim_crypto_df.NetHashesPerSecond.astype(float)
slim_crypto_df['TotalCoinSupply'] = slim_crypto_df.TotalCoinSupply.astype(float)
slim_crypto_df['TotalCoinsMined'] = slim_crypto_df.TotalCoinsMined.astype(float)

In [34]:
slim_crypto_df.dtypes

Algorithm              object
BlockNumber           float64
BlockReward           float64
BlockTime             float64
FullyPremined         float64
Name                   object
NetHashesPerSecond    float64
ProofType              object
Symbol                 object
TotalCoinSupply       float64
TotalCoinsMined       float64
dtype: object

In [39]:
# Creating dummies variables for text features
slim_crypto_encoded = pd.get_dummies(data=slim_crypto_df, columns=['Name', 'Algorithm', 'ProofType', 'Symbol'])

In [40]:
# Standardizing data
slim_crypto_df_scaled = StandardScaler().fit_transform(slim_crypto_encoded)
slim_crypto_df_scaled


array([[-0.22456036, -0.10157802, -0.0756658 , ..., -0.04360207,
        -0.04360207, -0.04360207],
       [ 0.35771367, -0.10157802, -0.07566524, ..., -0.04360207,
        -0.04360207, -0.04360207],
       [-0.26580412, -0.10157802, -0.0756658 , ..., -0.04360207,
        -0.04360207, -0.04360207],
       ...,
       [-0.26580412, -0.10027514, -0.07566563, ..., -0.04360207,
        -0.04360207, -0.04360207],
       [ 0.0534773 , -0.10056613, -0.0756645 , ..., -0.04360207,
        -0.04360207, -0.04360207],
       [-0.08538305, -0.09832082, -0.0756644 , ..., -0.04360207,
        -0.04360207, -0.04360207]])

### Reducing Dimensions Using PCA

In [0]:
# Using PCA to reduce dimension to 3 principal components



In [0]:
# Creating a DataFrame with the principal components data



### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [0]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:


# Create the Elbow Curve using hvPlot



Running K-Means with `k=<you best value for k here>`

In [0]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Creating a new DataFrame including predicted clusters and cryptocurrencies features



### Visualizing Results

#### 3D-Scatter with Clusters

In [0]:
# Creating a 3D-Scatter with the PCA data and the clusters



#### Table of Tradable Cryptocurrencies

In [0]:
# Table with tradable cryptos



#### Scatter Plot with Tradable Cryptocurrencies

In [0]:
# Scaling data to create the scatter plot



In [0]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

